In [ ]:
# Final code:
import authToken
import wbsoc
import time
import redis
import Save
import json
import os
import dotenv
import avgParser

testing = True

def main_run():
    token = authToken.AutoLogin('test').get_access_token()
    r = redis.Redis(host="localhost",port="6379",db=0)
    symb = wbsoc.Symbol(r,token,testing)
    depth = wbsoc.Depth(r,token,testing)
    symb.connect()
    depth.connect()
    symb.subscribe()
    depth.subscribe()
    time.sleep(5)
    print("its time to end!")
    depth.unsubscribe()
    symb.unsubscribe()
    symb.close()
    depth.close()


# test this bad boy. 
# saves all incoming data to csv files :)
def csvWorker(directory): # should be on a separate thread. 
    dotenv.load_dotenv()
    r = redis.Redis(host="localhost",port="6379",db=0)
    stonksList = json.loads(os.getenv("STOCKS"))["TEST"] if testing else json.reads(os.readenv("STOCKS"))["REAL"]
    worker = Save.csv(directory,testing)
    stonks = {stonk.split('-')[0]:'$' for stonk in stonksList}
    while True:
        for stream in r.xread(stonks):
            for uncoded_msg in stream[1]:
                try:
                    worker.save_msg({key.decode('utf-8'): value.decode('utf-8') for key, value in uncoded_msg[1].items()})
                except Exception:
                    break

    

def avgParserWorker(directory):
    r = redis.Redis(host="localhost",port="6379",db=0)
    stonksList = json.loads(os.getenv("STOCKS"))["TEST"] if testing else json.reads(os.readenv("STOCKS"))["REAL"]
    stonks = {stonk.split('-')[0]:'$' for stonk in stonksList}
    while r.get('end')!='true':
        data = r.xread(stonks)
        for stream in data:
            for uncoded_msg in stream[1]:
                msg = {key.decode('utf-8'): value.decode('utf-8') for key, value in uncoded_msg[1].items()} # decoding message
                parsed_msg = avgParser.parseMsg(r,msg)
                avgParser.to_csv(parsed_msg)

def endDay(): # clears cache. 
    stonksList = json.loads(os.getenv("STOCKS"))["TEST"] if testing else json.reads(os.readenv("STOCKS"))["REAL"]
    r = redis.Redis(host="localhost",port="6379",db=0)
    for stonk in stonksList:
        r.xtrim(stonk)

In [1]:
import authToken


In [1]:
import os
from dotenv import load_dotenv
import json

In [2]:
load_dotenv()

True

In [4]:
os.getenv("DATA_FIELDS")


'{"depth":["bid_price1", "bid_price2", "bid_price3", "bid_price4", "bid_price5", "ask_price1", "ask_price2", "ask_price3", "ask_price4", "ask_price5", "bid_size1", "bid_size2", "bid_size3", "bid_size4", "bid_size5", "ask_size1", "ask_size2", "ask_size3", "ask_size4", "ask_size5", "bid_order1", "bid_order2", "bid_order3", "bid_order4", "bid_order5", "ask_order1", "ask_order2", "ask_order3", "ask_order4", "ask_order5", "type", "symbol"],"symbol":["ltp", "vol_traded_today", "last_traded_time", "exch_feed_time", "bid_size", "ask_size", "bid_price", "ask_price", "last_traded_qty", "tot_buy_qty", "tot_sell_qty", "avg_trade_price", "low_price", "high_price", "lower_ckt", "upper_ckt", "open_price", "prev_close_price", "type", "symbol", "ch", "chp"]}'

In [8]:
json.loads(os.getenv("DATA_FIELDS"))['symbol']

['ltp',
 'vol_traded_today',
 'last_traded_time',
 'exch_feed_time',
 'bid_size',
 'ask_size',
 'bid_price',
 'ask_price',
 'last_traded_qty',
 'tot_buy_qty',
 'tot_sell_qty',
 'avg_trade_price',
 'low_price',
 'high_price',
 'lower_ckt',
 'upper_ckt',
 'open_price',
 'prev_close_price',
 'type',
 'symbol',
 'ch',
 'chp']

In [8]:
json.loads(os.getenv("STOCKS"))["testing"]

['BSE:TCS-A', 'NSE:TCS-EQ']

# TESTING authToken to make sure it works

In [6]:
import authToken

In [7]:
token = authToken.AutoLogin('test').get_access_token()

no neeed to validate :)
auth_code obtained!
200
********************************************************************************
success


In [1]:
token ='eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJhcGkuZnllcnMuaW4iLCJpYXQiOjE3MzQ2Nzg0NTksImV4cCI6MTczNDc0MTAzOSwibmJmIjoxNzM0Njc4NDU5LCJhdWQiOlsieDowIiwieDoxIiwieDoyIiwiZDoxIiwiZDoyIiwieDoxIiwieDowIl0sInN1YiI6ImFjY2Vzc190b2tlbiIsImF0X2hhc2giOiJnQUFBQUFCblpSZTdNaHYycEpqdm9jNnRvNnpKeDNsZWt6VEIydG5ES2t2aFBiSEpwSWZOZm9UMUxQYVI0OGptNUVSTmZhb3NnQWl3eVluOVlvNjhCS0xnYTV3M2o1VzdZQVBxbGI1bDNaclRWYzg2VUhiMFJDRT0iLCJkaXNwbGF5X25hbWUiOiJNQUxMRVBBTExJIEdBSlVMQSBHVVJVIFNBSSBQUkFTQUQiLCJvbXMiOiJLMSIsImhzbV9rZXkiOiJiNGM5NzE0ZmY3ZjViYTBjNWYzMGRlYTEyOWIxY2Q1MGFkYjZkOTllZjk1MzhhMWVhMTVjMmUyYiIsImZ5X2lkIjoiWU0wODkyNyIsImFwcFR5cGUiOjEwMCwicG9hX2ZsYWciOiJOIn0.hsJ6leTo3Omxyi8cNm17ic5BnE4QoDcIxfFe0J1WSdA'

### next step: sending all my inputs to a singular cache (and saving them to csvs :D )

In [2]:
import wbsoc
import time
import redis

In [3]:
r = redis.Redis(host="localhost",port="6379",db=0)
symb = wbsoc.Symbol(r,token,True)
depth = wbsoc.Depth(r,token,True)

In [4]:

symb.connect()
depth.connect()

Response: {'type': 'cn', 'code': 200, 'message': 'Authentication done', 's': 'ok'}
connection opened
Response: {'type': 'cn', 'code': 200, 'message': 'Authentication done', 's': 'ok'}
connection opened


In [5]:
symb.subscribe()
depth.subscribe()
time.sleep(5)
print("its time to end!")
depth.unsubscribe()
symb.unsubscribe()
symb.close()
depth.close()


Response: {'type': 'ful', 'code': 200, 'message': 'Full Mode On', 's': 'ok'}
Response: {'type': 'ful', 'code': 200, 'message': 'Full Mode On', 's': 'ok'}
Response: {'type': 'sub', 'code': 200, 'message': 'Subscribed', 's': 'ok'}
Response: {'ltp': 4171.0, 'vol_traded_today': 2854335, 'last_traded_time': 1734687083, 'exch_feed_time': 1734687083, 'bid_size': 22, 'ask_size': 113, 'bid_price': 4169.75, 'ask_price': 4171.0, 'last_traded_qty': 4, 'tot_buy_qty': 88329, 'tot_sell_qty': 179327, 'avg_trade_price': 4270.69, 'low_price': 4160.8, 'high_price': 4382.0, 'lower_ckt': 0, 'upper_ckt': 0, 'open_price': 4329.0, 'prev_close_price': 4271.9, 'type': 'sf', 'symbol': 'NSE:TCS-EQ', 'ch': -100.9, 'chp': -2.3619}
Response: {'ltp': 4173.45, 'vol_traded_today': 71354, 'last_traded_time': 1734687075, 'exch_feed_time': 1734687082, 'bid_size': 15, 'ask_size': 22, 'bid_price': 4170.3, 'ask_price': 4172.45, 'last_traded_qty': 6, 'tot_buy_qty': 21355, 'tot_sell_qty': 270670, 'avg_trade_price': 4239.08, 'l

its time to end!
unsubscribed from depth
unsubscribed from symbol


In [4]:
r.xrange('NSE:TCS','-','+')

[(b'1734686965307-0',
  {b'ltp': b'4181.5',
   b'vol_traded_today': b'2776701',
   b'last_traded_time': b'1734686963',
   b'exch_feed_time': b'1734686964',
   b'bid_size': b'22',
   b'ask_size': b'233',
   b'bid_price': b'4181.15',
   b'ask_price': b'4181.5',
   b'last_traded_qty': b'1',
   b'tot_buy_qty': b'92749',
   b'tot_sell_qty': b'186371',
   b'avg_trade_price': b'4273.32',
   b'low_price': b'4160.8',
   b'high_price': b'4382.0',
   b'lower_ckt': b'0',
   b'upper_ckt': b'0',
   b'open_price': b'4329.0',
   b'prev_close_price': b'4271.9',
   b'type': b'sf',
   b'symbol': b'NSE:TCS-EQ',
   b'ch': b'-90.4',
   b'chp': b'-2.1162'}),
 (b'1734686965546-0',
  {b'ltp': b'4181.5',
   b'vol_traded_today': b'2776795',
   b'last_traded_time': b'1734686964',
   b'exch_feed_time': b'1734686964',
   b'bid_size': b'4',
   b'ask_size': b'139',
   b'bid_price': b'4180.95',
   b'ask_price': b'4181.4',
   b'last_traded_qty': b'15',
   b'tot_buy_qty': b'91448',
   b'tot_sell_qty': b'186693',
   b'av

In [47]:
#r.xread({"NSE:TCS":0})[0][1][0][1]
len(r.xread({"NSE:TCS":0})[0][1])

33

In [49]:
r.xread({"NSE:TCS":0})[0][1][0][1]

{b'ltp': b'4181.5',
 b'vol_traded_today': b'2776701',
 b'last_traded_time': b'1734686963',
 b'exch_feed_time': b'1734686964',
 b'bid_size': b'22',
 b'ask_size': b'233',
 b'bid_price': b'4181.15',
 b'ask_price': b'4181.5',
 b'last_traded_qty': b'1',
 b'tot_buy_qty': b'92749',
 b'tot_sell_qty': b'186371',
 b'avg_trade_price': b'4273.32',
 b'low_price': b'4160.8',
 b'high_price': b'4382.0',
 b'lower_ckt': b'0',
 b'upper_ckt': b'0',
 b'open_price': b'4329.0',
 b'prev_close_price': b'4271.9',
 b'type': b'sf',
 b'symbol': b'NSE:TCS-EQ',
 b'ch': b'-90.4',
 b'chp': b'-2.1162'}

In [ ]:
import redis
r = redis.Redis(host="localhost",port="6379",db=0)

#msg = [{key.decode('utf-8'): value.decode('utf-8') for key, value in x[0][1][0][1].items()} for x in r.xread({"NSE:TCS":0})]

msg = []
for stream in r.xread({"NSE:TCS":0}):
    for uncoded_msg in stream[1]:
        {key.decode('utf-8'): value.decode('utf-8') for key, value in uncoded_msg[1].items()} 
        

In [2]:
import tstParser
import redis
r = redis.Redis(host="localhost",port="6379",db=0)
t = redis.Redis(host="localhost",port="6379",db=1)
data = r.xread({"NSE:TCS":0})
for stream in data:
    for uncoded_msg in stream[1]:
        msg = {key.decode('utf-8'): value.decode('utf-8') for key, value in uncoded_msg[1].items()} # decoding message
        parsed_msg = tstParser.parseMsg(t,msg)
        print(parsed_msg)
        tstParser.to_csv(parsed_msg,'./data')
        #print('saved to file!')

None
preparing to save file!
	 no file found ;(
None
preparing to save file!
	 no file found ;(
None
preparing to save file!
	 no file found ;(
None
preparing to save file!
	 no file found ;(
None
preparing to save file!
	 no file found ;(
None
preparing to save file!
	 no file found ;(
None
preparing to save file!
	 no file found ;(
None
preparing to save file!
	 no file found ;(
None
preparing to save file!
	 no file found ;(
None
preparing to save file!
	 no file found ;(
None
preparing to save file!
	 no file found ;(
None
preparing to save file!
	 no file found ;(
None
preparing to save file!
	 no file found ;(
None
preparing to save file!
	 no file found ;(
None
preparing to save file!
	 no file found ;(
None
preparing to save file!
	 no file found ;(
None
preparing to save file!
	 no file found ;(
None
preparing to save file!
	 no file found ;(
None
preparing to save file!
	 no file found ;(
None
preparing to save file!
	 no file found ;(
None
preparing to save file!
	 no file f

lets try putting the connections on dedicated threads 

In [5]:
t.flushdb()

True

In [17]:
r.xread({"NSE:TCS":0})

[[b'NSE:TCS',
  [(b'1734768460305-0',
    {b'ltp': b'4181.5',
     b'vol_traded_today': b'2776701',
     b'last_traded_time': b'1734686963',
     b'exch_feed_time': b'1734686964',
     b'bid_size': b'22',
     b'ask_size': b'233',
     b'bid_price': b'4181.15',
     b'ask_price': b'4181.5',
     b'last_traded_qty': b'1',
     b'tot_buy_qty': b'92749',
     b'tot_sell_qty': b'186371',
     b'avg_trade_price': b'4273.32',
     b'low_price': b'4160.8',
     b'high_price': b'4382.0',
     b'lower_ckt': b'0',
     b'upper_ckt': b'0',
     b'open_price': b'4329.0',
     b'prev_close_price': b'4271.9',
     b'type': b'sf',
     b'symbol': b'NSE:TCS-EQ',
     b'ch': b'-90.4',
     b'chp': b'-2.1162'}),
   (b'1734768460306-0',
    {b'ltp': b'4181.5',
     b'vol_traded_today': b'2776795',
     b'last_traded_time': b'1734686964',
     b'exch_feed_time': b'1734686964',
     b'bid_size': b'4',
     b'ask_size': b'139',
     b'bid_price': b'4180.95',
     b'ask_price': b'4181.4',
     b'last_traded_

In [14]:
data = r.xread({"NSE:TCS":0})
for stream in data:
    for uncoded_msg in stream[1]:
        msg = {key.decode('utf-8'): value.decode('utf-8') for key, value in uncoded_msg[1].items()} # decoding message
        print(msg)

{'ltp': '4181.5', 'vol_traded_today': '2776701', 'last_traded_time': '1734686963', 'exch_feed_time': '1734686964', 'bid_size': '22', 'ask_size': '233', 'bid_price': '4181.15', 'ask_price': '4181.5', 'last_traded_qty': '1', 'tot_buy_qty': '92749', 'tot_sell_qty': '186371', 'avg_trade_price': '4273.32', 'low_price': '4160.8', 'high_price': '4382.0', 'lower_ckt': '0', 'upper_ckt': '0', 'open_price': '4329.0', 'prev_close_price': '4271.9', 'type': 'sf', 'symbol': 'NSE:TCS-EQ', 'ch': '-90.4', 'chp': '-2.1162'}
{'ltp': '4181.5', 'vol_traded_today': '2776795', 'last_traded_time': '1734686964', 'exch_feed_time': '1734686964', 'bid_size': '4', 'ask_size': '139', 'bid_price': '4180.95', 'ask_price': '4181.4', 'last_traded_qty': '15', 'tot_buy_qty': '91448', 'tot_sell_qty': '186693', 'avg_trade_price': '4273.32', 'low_price': '4160.8', 'high_price': '4382.0', 'lower_ckt': '0', 'upper_ckt': '0', 'open_price': '4329.0', 'prev_close_price': '4271.9', 'type': 'sf', 'symbol': 'NSE:TCS-EQ', 'ch': '-90.

In [10]:
import multiprocessing
import threading
import wbsoc

import redis

In [11]:
def connectAndDisconnect(token,type,testing:bool,sleep_time,redis_client):
    stockObj =wbsoc.Depth(token,testing) if type=="depth" else wbsoc.Symbol(redis_client,token,testing)
    stockObj.connect()
    stockObj.subscribe()
    time.sleep(sleep_time)
    stockObj.unsubscribe()
    stockObj.close()


In [12]:
r = redis.Redis(host="localhost",port="6379",db=0)
uno = threading.Thread(target=connectAndDisconnect,args=(token,"symbol",True,5,r))
dos = threading.Thread(target=connectAndDisconnect,args=(token,"depth",True,5,r))

Response: {'type': 'cn', 'code': 200, 'message': 'Authentication done', 's': 'ok'}
Response: {'type': 'ful', 'code': 200, 'message': 'Full Mode On', 's': 'ok'}
connection openedconnection opened

Response: {'type': 'sub', 'code': 200, 'message': 'Subscribed', 's': 'ok'}
Response: {'type': 'sub', 'code': 200, 'message': 'Subscribed', 's': 'ok'}
Response: {'bid_price1': 4171.95, 'bid_price2': 4171.15, 'bid_price3': 4171.1, 'bid_price4': 4171.05, 'bid_price5': 4171.0, 'ask_price1': 4172.5, 'ask_price2': 4172.55, 'ask_price3': 4172.9, 'ask_price4': 4172.95, 'ask_price5': 4173.15, 'bid_size1': 54, 'bid_size2': 13, 'bid_size3': 121, 'bid_size4': 1, 'bid_size5': 28, 'ask_size1': 129, 'ask_size2': 15, 'ask_size3': 11, 'ask_size4': 3, 'ask_size5': 15, 'bid_order1': 1, 'bid_order2': 1, 'bid_order3': 1, 'bid_order4': 1, 'bid_order5': 3, 'ask_order1': 1, 'ask_order2': 3, 'ask_order3': 1, 'ask_order4': 1, 'ask_order5': 1, 'type': 'dp', 'symbol': 'NSE:TCS-EQ'}
Response: {'ltp': 4172.5, 'vol_traded_t

# alrighty we're gettting both simultaneously without any extra threads and other bs :D


### next steps: 
    - design redis stream 
        "BSE"
        "NSE:
    - pump messages to redis stream 

In [1]:
import Save

In [2]:
test = Save.csv("./data",True)

In [3]:
test.initialise()

In [4]:
test.save_files({'ltp': 4172.5, 'vol_traded_today': 2467852, 'last_traded_time': 1734683133, 'exch_feed_time': 1734683134, 'bid_size': 54, 'ask_size': 129, 'bid_price': 4171.95, 'ask_price': 4172.5, 'last_traded_qty': 3, 'tot_buy_qty': 97283, 'tot_sell_qty': 188301, 'avg_trade_price': 4284.66, 'low_price': 4160.8, 'high_price': 4382.0, 'lower_ckt': 0, 'upper_ckt': 0, 'open_price': 4329.0, 'prev_close_price': 4271.9, 'type': 'sf', 'symbol': 'NSE:TCS-EQ', 'ch': -99.4, 'chp': -2.3268})

In [3]:
r.set('ltp',4172.5)

True

In [13]:
float(r.get('ltp'))

4172.5

In [14]:
r.get('NSE:TCS-bid')

In [1]:
import Save

In [2]:
Save.csv('./data',True).initialise()

In [7]:
import os
d = os.path.join('./data','hi','bob')
os.makedirs(d)

In [8]:
hello = [{'ltp': '4181.5', 'vol_traded_today': '2776701', 'last_traded_time': '1734686963', 'exch_feed_time': '1734686964', 'bid_size': '22', 'ask_size': '233', 'bid_price': '4181.15', 'ask_price': '4181.5', 'last_traded_qty': '1', 'tot_buy_qty': '92749', 'tot_sell_qty': '186371', 'avg_trade_price': '4273.32', 'low_price': '4160.8', 'high_price': '4382.0', 'lower_ckt': '0', 'upper_ckt': '0', 'open_price': '4329.0', 'prev_close_price': '4271.9', 'type': 'sf', 'symbol': 'NSE:TCS-EQ', 'ch': '-90.4', 'chp': '-2.1162'},
{'ltp': '4181.5', 'vol_traded_today': '2776795', 'last_traded_time': '1734686964', 'exch_feed_time': '1734686964', 'bid_size': '4', 'ask_size': '139', 'bid_price': '4180.95', 'ask_price': '4181.4', 'last_traded_qty': '15', 'tot_buy_qty': '91448', 'tot_sell_qty': '186693', 'avg_trade_price': '4273.32', 'low_price': '4160.8', 'high_price': '4382.0', 'lower_ckt': '0', 'upper_ckt': '0', 'open_price': '4329.0', 'prev_close_price': '4271.9', 'type': 'sf', 'symbol': 'NSE:TCS-EQ', 'ch': '-90.4', 'chp': '-2.1162'},
{'ltp': '4181.35', 'vol_traded_today': '2776882', 'last_traded_time': '1734686965', 'exch_feed_time': '1734686965', 'bid_size': '4', 'ask_size': '52', 'bid_price': '4180.95', 'ask_price': '4181.35', 'last_traded_qty': '44', 'tot_buy_qty': '92727', 'tot_sell_qty': '186686', 'avg_trade_price': '4273.32', 'low_price': '4160.8', 'high_price': '4382.0', 'lower_ckt': '0', 'upper_ckt': '0', 'open_price': '4329.0', 'prev_close_price': '4271.9', 'type': 'sf', 'symbol': 'NSE:TCS-EQ', 'ch': '-90.55', 'chp': '-2.1197'},
{'ltp': '4182.65', 'vol_traded_today': '2777196', 'last_traded_time': '1734686965', 'exch_feed_time': '1734686966', 'bid_size': '3', 'ask_size': '5', 'bid_price': '4181.35', 'ask_price': '4181.4', 'last_traded_qty': '3', 'tot_buy_qty': '94707', 'tot_sell_qty': '183032', 'avg_trade_price': '4273.31', 'low_price': '4160.8', 'high_price': '4382.0', 'lower_ckt': '0', 'upper_ckt': '0', 'open_price': '4329.0', 'prev_close_price': '4271.9', 'type': 'sf', 'symbol': 'NSE:TCS-EQ', 'ch': '-89.25', 'chp': '-2.0892'},
{'ltp': '4183.0', 'vol_traded_today': '2777487', 'last_traded_time': '1734686966', 'exch_feed_time': '1734686966', 'bid_size': '6', 'ask_size': '9', 'bid_price': '4181.6', 'ask_price': '4182.9', 'last_traded_qty': '12', 'tot_buy_qty': '95621', 'tot_sell_qty': '181295', 'avg_trade_price': '4273.3', 'low_price': '4160.8', 'high_price': '4382.0', 'lower_ckt': '0', 'upper_ckt': '0', 'open_price': '4329.0', 'prev_close_price': '4271.9', 'type': 'sf', 'symbol': 'NSE:TCS-EQ', 'ch': '-88.9', 'chp': '-2.081'},
{'ltp': '4182.45', 'vol_traded_today': '2777946', 'last_traded_time': '1734686967', 'exch_feed_time': '1734686967', 'bid_size': '15', 'ask_size': '29', 'bid_price': '4181.85', 'ask_price': '4182.95', 'last_traded_qty': '71', 'tot_buy_qty': '95586', 'tot_sell_qty': '181385', 'avg_trade_price': '4273.28', 'low_price': '4160.8', 'high_price': '4382.0', 'lower_ckt': '0', 'upper_ckt': '0', 'open_price': '4329.0', 'prev_close_price': '4271.9', 'type': 'sf', 'symbol': 'NSE:TCS-EQ', 'ch': '-89.45', 'chp': '-2.0939'},
{'ltp': '4182.95', 'vol_traded_today': '2778027', 'last_traded_time': '1734686967', 'exch_feed_time': '1734686967', 'bid_size': '15', 'ask_size': '6', 'bid_price': '4181.95', 'ask_price': '4182.95', 'last_traded_qty': '12', 'tot_buy_qty': '93658', 'tot_sell_qty': '184696', 'avg_trade_price': '4273.28', 'low_price': '4160.8', 'high_price': '4382.0', 'lower_ckt': '0', 'upper_ckt': '0', 'open_price': '4329.0', 'prev_close_price': '4271.9', 'type': 'sf', 'symbol': 'NSE:TCS-EQ', 'ch': '-88.95', 'chp': '-2.0822'},
{'ltp': '4182.95', 'vol_traded_today': '2778140', 'last_traded_time': '1734686968', 'exch_feed_time': '1734686968', 'bid_size': '2', 'ask_size': '7', 'bid_price': '4181.9', 'ask_price': '4182.95', 'last_traded_qty': '20', 'tot_buy_qty': '93966', 'tot_sell_qty': '185528', 'avg_trade_price': '4273.27', 'low_price': '4160.8', 'high_price': '4382.0', 'lower_ckt': '0', 'upper_ckt': '0', 'open_price': '4329.0', 'prev_close_price': '4271.9', 'type': 'sf', 'symbol': 'NSE:TCS-EQ', 'ch': '-88.95', 'chp': '-2.0822'},
{'ltp': '4183.0', 'vol_traded_today': '2778356', 'last_traded_time': '1734686968', 'exch_feed_time': '1734686968', 'bid_size': '9', 'ask_size': '37', 'bid_price': '4182.0', 'ask_price': '4183.0', 'last_traded_qty': '13', 'tot_buy_qty': '93879', 'tot_sell_qty': '183759', 'avg_trade_price': '4273.27', 'low_price': '4160.8', 'high_price': '4382.0', 'lower_ckt': '0', 'upper_ckt': '0', 'open_price': '4329.0', 'prev_close_price': '4271.9', 'type': 'sf', 'symbol': 'NSE:TCS-EQ', 'ch': '-88.9', 'chp': '-2.081'},
{'ltp': '4183.0', 'vol_traded_today': '2778356', 'last_traded_time': '1734686968', 'exch_feed_time': '1734686969', 'bid_size': '9', 'ask_size': '37', 'bid_price': '4182.0', 'ask_price': '4183.0', 'last_traded_qty': '23', 'tot_buy_qty': '93879', 'tot_sell_qty': '183759', 'avg_trade_price': '4273.27', 'low_price': '4160.8', 'high_price': '4382.0', 'lower_ckt': '0', 'upper_ckt': '0', 'open_price': '4329.0', 'prev_close_price': '4271.9', 'type': 'sf', 'symbol': 'NSE:TCS-EQ', 'ch': '-88.9', 'chp': '-2.081'},
{'ltp': '4183.35', 'vol_traded_today': '2778776', 'last_traded_time': '1734686969', 'exch_feed_time': '1734686969', 'bid_size': '1', 'ask_size': '9', 'bid_price': '4183.35', 'ask_price': '4184.0', 'last_traded_qty': '8', 'tot_buy_qty': '95270', 'tot_sell_qty': '178294', 'avg_trade_price': '4273.25', 'low_price': '4160.8', 'high_price': '4382.0', 'lower_ckt': '0', 'upper_ckt': '0', 'open_price': '4329.0', 'prev_close_price': '4271.9', 'type': 'sf', 'symbol': 'NSE:TCS-EQ', 'ch': '-88.55', 'chp': '-2.0728'},
{'ltp': '4183.3', 'vol_traded_today': '2778916', 'last_traded_time': '1734686969', 'exch_feed_time': '1734686969', 'bid_size': '8', 'ask_size': '46', 'bid_price': '4182.3', 'ask_price': '4183.35', 'last_traded_qty': '38', 'tot_buy_qty': '95496', 'tot_sell_qty': '179778', 'avg_trade_price': '4273.25', 'low_price': '4160.8', 'high_price': '4382.0', 'lower_ckt': '0', 'upper_ckt': '0', 'open_price': '4329.0', 'prev_close_price': '4271.9', 'type': 'sf', 'symbol': 'NSE:TCS-EQ', 'ch': '-88.6', 'chp': '-2.074'},
{'ltp': '4182.65', 'vol_traded_today': '2779019', 'last_traded_time': '1734686970', 'exch_feed_time': '1734686970', 'bid_size': '8', 'ask_size': '118', 'bid_price': '4181.55', 'ask_price': '4182.65', 'last_traded_qty': '1', 'tot_buy_qty': '95756', 'tot_sell_qty': '185183', 'avg_trade_price': '4273.25', 'low_price': '4160.8', 'high_price': '4382.0', 'lower_ckt': '0', 'upper_ckt': '0', 'open_price': '4329.0', 'prev_close_price': '4271.9', 'type': 'sf', 'symbol': 'NSE:TCS-EQ', 'ch': '-89.25', 'chp': '-2.0892'},
{'ltp': '4171.0', 'vol_traded_today': '2854335', 'last_traded_time': '1734687083', 'exch_feed_time': '1734687083', 'bid_size': '22', 'ask_size': '113', 'bid_price': '4169.75', 'ask_price': '4171.0', 'last_traded_qty': '4', 'tot_buy_qty': '88329', 'tot_sell_qty': '179327', 'avg_trade_price': '4270.69', 'low_price': '4160.8', 'high_price': '4382.0', 'lower_ckt': '0', 'upper_ckt': '0', 'open_price': '4329.0', 'prev_close_price': '4271.9', 'type': 'sf', 'symbol': 'NSE:TCS-EQ', 'ch': '-100.9', 'chp': '-2.3619'},
{'bid_price1': '4169.75', 'bid_price2': '4169.7', 'bid_price3': '4169.65', 'bid_price4': '4169.55', 'bid_price5': '4169.5', 'ask_price1': '4171.0', 'ask_price2': '4171.05', 'ask_price3': '4171.1', 'ask_price4': '4171.15', 'ask_price5': '4171.2', 'bid_size1': '22', 'bid_size2': '4', 'bid_size3': '30', 'bid_size4': '2', 'bid_size5': '23', 'ask_size1': '113', 'ask_size2': '11', 'ask_size3': '1901', 'ask_size4': '1240', 'ask_size5': '153', 'bid_order1': '1', 'bid_order2': '2', 'bid_order3': '2', 'bid_order4': '2', 'bid_order5': '3', 'ask_order1': '5', 'ask_order2': '1', 'ask_order3': '3', 'ask_order4': '1', 'ask_order5': '4', 'type': 'dp', 'symbol': 'NSE:TCS-EQ'},
{'ltp': '4169.75', 'vol_traded_today': '2854335', 'last_traded_time': '1734687083', 'exch_feed_time': '1734687084', 'bid_size': '22', 'ask_size': '113', 'bid_price': '4169.75', 'ask_price': '4171.0', 'last_traded_qty': '5', 'tot_buy_qty': '88329', 'tot_sell_qty': '179327', 'avg_trade_price': '4270.69', 'low_price': '4160.8', 'high_price': '4382.0', 'lower_ckt': '0', 'upper_ckt': '0', 'open_price': '4329.0', 'prev_close_price': '4271.9', 'type': 'sf', 'symbol': 'NSE:TCS-EQ', 'ch': '-102.15', 'chp': '-2.3912'},
{'bid_price1': '4169.25', 'bid_price2': '4169.2', 'bid_price3': '4169.15', 'bid_price4': '4169.05', 'bid_price5': '4169.0', 'ask_price1': '4170.45', 'ask_price2': '4170.5', 'ask_price3': '4170.8', 'ask_price4': '4170.9', 'ask_price5': '4171.0', 'bid_size1': '5', 'bid_size2': '222', 'bid_size3': '1', 'bid_size4': '51', 'bid_size5': '189', 'ask_size1': '7', 'ask_size2': '11', 'ask_size3': '14', 'ask_size4': '37', 'ask_size5': '112', 'bid_order1': '1', 'bid_order2': '2', 'bid_order3': '1', 'bid_order4': '2', 'bid_order5': '5', 'ask_order1': '1', 'ask_order2': '2', 'ask_order3': '1', 'ask_order4': '1', 'ask_order5': '5', 'type': 'dp', 'symbol': 'NSE:TCS-EQ'},
{'ltp': '4170.45', 'vol_traded_today': '2854588', 'last_traded_time': '1734687084', 'exch_feed_time': '1734687084', 'bid_size': '5', 'ask_size': '7', 'bid_price': '4169.25', 'ask_price': '4170.45', 'last_traded_qty': '2', 'tot_buy_qty': '88355', 'tot_sell_qty': '179546', 'avg_trade_price': '4270.68', 'low_price': '4160.8', 'high_price': '4382.0', 'lower_ckt': '0', 'upper_ckt': '0', 'open_price': '4329.0', 'prev_close_price': '4271.9', 'type': 'sf', 'symbol': 'NSE:TCS-EQ', 'ch': '-101.45', 'chp': '-2.3748'},
{'ltp': '4169.75', 'vol_traded_today': '2854588', 'last_traded_time': '1734687084', 'exch_feed_time': '1734687085', 'bid_size': '5', 'ask_size': '7', 'bid_price': '4169.25', 'ask_price': '4170.45', 'last_traded_qty': '12', 'tot_buy_qty': '88355', 'tot_sell_qty': '179546', 'avg_trade_price': '4270.68', 'low_price': '4160.8', 'high_price': '4382.0', 'lower_ckt': '0', 'upper_ckt': '0', 'open_price': '4329.0', 'prev_close_price': '4271.9', 'type': 'sf', 'symbol': 'NSE:TCS-EQ', 'ch': '-102.15', 'chp': '-2.3912'},
{'ltp': '4169.75', 'vol_traded_today': '2854588', 'last_traded_time': '1734687084', 'exch_feed_time': '1734687085', 'bid_size': '5', 'ask_size': '7', 'bid_price': '4169.25', 'ask_price': '4170.45', 'last_traded_qty': '5', 'tot_buy_qty': '88355', 'tot_sell_qty': '179546', 'avg_trade_price': '4270.68', 'low_price': '4160.8', 'high_price': '4382.0', 'lower_ckt': '0', 'upper_ckt': '0', 'open_price': '4329.0', 'prev_close_price': '4271.9', 'type': 'sf', 'symbol': 'NSE:TCS-EQ', 'ch': '-102.15', 'chp': '-2.3912'},
{'bid_price1': '4168.9', 'bid_price2': '4168.8', 'bid_price3': '4168.75', 'bid_price4': '4168.55', 'bid_price5': '4168.45', 'ask_price1': '4169.0', 'ask_price2': '4169.65', 'ask_price3': '4169.8', 'ask_price4': '4169.85', 'ask_price5': '4169.95', 'bid_size1': '719', 'bid_size2': '22', 'bid_size3': '2', 'bid_size4': '1', 'bid_size5': '56', 'ask_size1': '3', 'ask_size2': '9', 'ask_size3': '17', 'ask_size4': '22', 'ask_size5': '27', 'bid_order1': '1', 'bid_order2': '2', 'bid_order3': '2', 'bid_order4': '1', 'bid_order5': '1', 'ask_order1': '1', 'ask_order2': '2', 'ask_order3': '1', 'ask_order4': '2', 'ask_order5': '1', 'type': 'dp', 'symbol': 'NSE:TCS-EQ'},
{'ltp': '4168.9', 'vol_traded_today': '2855310', 'last_traded_time': '1734687085', 'exch_feed_time': '1734687085', 'bid_size': '719', 'ask_size': '3', 'bid_price': '4168.9', 'ask_price': '4169.0', 'last_traded_qty': '5', 'tot_buy_qty': '87436', 'tot_sell_qty': '177344', 'avg_trade_price': '4270.65', 'low_price': '4160.8', 'high_price': '4382.0', 'lower_ckt': '0', 'upper_ckt': '0', 'open_price': '4329.0', 'prev_close_price': '4271.9', 'type': 'sf', 'symbol': 'NSE:TCS-EQ', 'ch': '-103.0', 'chp': '-2.4111'},
{'ltp': '4169.65', 'vol_traded_today': '2855310', 'last_traded_time': '1734687085', 'exch_feed_time': '1734687086', 'bid_size': '719', 'ask_size': '3', 'bid_price': '4168.9', 'ask_price': '4169.0', 'last_traded_qty': '45', 'tot_buy_qty': '87436', 'tot_sell_qty': '177344', 'avg_trade_price': '4270.65', 'low_price': '4160.8', 'high_price': '4382.0', 'lower_ckt': '0', 'upper_ckt': '0', 'open_price': '4329.0', 'prev_close_price': '4271.9', 'type': 'sf', 'symbol': 'NSE:TCS-EQ', 'ch': '-102.25', 'chp': '-2.3935'},
{'bid_price1': '4169.8', 'bid_price2': '4168.7', 'bid_price3': '4168.65', 'bid_price4': '4168.6', 'bid_price5': '4168.55', 'ask_price1': '4170.3', 'ask_price2': '4170.35', 'ask_price3': '4170.45', 'ask_price4': '4170.5', 'ask_price5': '4170.55', 'bid_size1': '4', 'bid_size2': '2', 'bid_size3': '12', 'bid_size4': '173', 'bid_size5': '86', 'ask_size1': '42', 'ask_size2': '226', 'ask_size3': '63', 'ask_size4': '6', 'ask_size5': '7', 'bid_order1': '2', 'bid_order2': '1', 'bid_order3': '1', 'bid_order4': '2', 'bid_order5': '4', 'ask_order1': '3', 'ask_order2': '3', 'ask_order3': '2', 'ask_order4': '3', 'ask_order5': '1', 'type': 'dp', 'symbol': 'NSE:TCS-EQ'},
{'ltp': '4169.8', 'vol_traded_today': '2855502', 'last_traded_time': '1734687087', 'exch_feed_time': '1734687086', 'bid_size': '4', 'ask_size': '42', 'bid_price': '4169.8', 'ask_price': '4170.3', 'last_traded_qty': '2', 'tot_buy_qty': '87758', 'tot_sell_qty': '178008', 'avg_trade_price': '4270.64', 'low_price': '4160.8', 'high_price': '4382.0', 'lower_ckt': '0', 'upper_ckt': '0', 'open_price': '4329.0', 'prev_close_price': '4271.9', 'type': 'sf', 'symbol': 'NSE:TCS-EQ', 'ch': '-102.1', 'chp': '-2.39'},
{'ltp': '4168.6', 'vol_traded_today': '2855502', 'last_traded_time': '1734687087', 'exch_feed_time': '1734687087', 'bid_size': '4', 'ask_size': '42', 'bid_price': '4169.8', 'ask_price': '4170.3', 'last_traded_qty': '113', 'tot_buy_qty': '87758', 'tot_sell_qty': '178008', 'avg_trade_price': '4270.64', 'low_price': '4160.8', 'high_price': '4382.0', 'lower_ckt': '0', 'upper_ckt': '0', 'open_price': '4329.0', 'prev_close_price': '4271.9', 'type': 'sf', 'symbol': 'NSE:TCS-EQ', 'ch': '-103.3', 'chp': '-2.4181'},
{'ltp': '4170.35', 'vol_traded_today': '2855502', 'last_traded_time': '1734687087', 'exch_feed_time': '1734687087', 'bid_size': '4', 'ask_size': '42', 'bid_price': '4169.8', 'ask_price': '4170.3', 'last_traded_qty': '16', 'tot_buy_qty': '87758', 'tot_sell_qty': '178008', 'avg_trade_price': '4270.64', 'low_price': '4160.8', 'high_price': '4382.0', 'lower_ckt': '0', 'upper_ckt': '0', 'open_price': '4329.0', 'prev_close_price': '4271.9', 'type': 'sf', 'symbol': 'NSE:TCS-EQ', 'ch': '-101.55', 'chp': '-2.3772'},
{'bid_price1': '4169.8', 'bid_price2': '4168.5', 'bid_price3': '4168.45', 'bid_price4': '4168.4', 'bid_price5': '4168.35', 'ask_price1': '4170.0', 'ask_price2': '4170.3', 'ask_price3': '4170.35', 'ask_price4': '4170.45', 'ask_price5': '4170.5', 'bid_size1': '3', 'bid_size2': '12', 'bid_size3': '24', 'bid_size4': '728', 'bid_size5': '1', 'ask_size1': '46', 'ask_size2': '42', 'ask_size3': '208', 'ask_size4': '64', 'ask_size5': '2', 'bid_order1': '1', 'bid_order2': '1', 'bid_order3': '1', 'bid_order4': '2', 'bid_order5': '1', 'ask_order1': '4', 'ask_order2': '3', 'ask_order3': '2', 'ask_order4': '3', 'ask_order5': '2', 'type': 'dp', 'symbol': 'NSE:TCS-EQ'},
{'ltp': '4168.45', 'vol_traded_today': '2856443', 'last_traded_time': '1734687088', 'exch_feed_time': '1734687088', 'bid_size': '3', 'ask_size': '46', 'bid_price': '4169.8', 'ask_price': '4170.0', 'last_traded_qty': '31', 'tot_buy_qty': '86759', 'tot_sell_qty': '180629', 'avg_trade_price': '4270.61', 'low_price': '4160.8', 'high_price': '4382.0', 'lower_ckt': '0', 'upper_ckt': '0', 'open_price': '4329.0', 'prev_close_price': '4271.9', 'type': 'sf', 'symbol': 'NSE:TCS-EQ', 'ch': '-103.45', 'chp': '-2.4216'},
{'bid_price1': '4168.7', 'bid_price2': '4168.6', 'bid_price3': '4168.5', 'bid_price4': '4168.45', 'bid_price5': '4168.4', 'ask_price1': '4170.25', 'ask_price2': '4170.3', 'ask_price3': '4170.35', 'ask_price4': '4170.45', 'ask_price5': '4170.5', 'bid_size1': '238', 'bid_size2': '10', 'bid_size3': '210', 'bid_size4': '355', 'bid_size5': '959', 'ask_size1': '12', 'ask_size2': '27', 'ask_size3': '64', 'ask_size4': '63', 'ask_size5': '2', 'bid_order1': '3', 'bid_order2': '1', 'bid_order3': '1', 'bid_order4': '3', 'bid_order5': '4', 'ask_order1': '1', 'ask_order2': '3', 'ask_order3': '1', 'ask_order4': '2', 'ask_order5': '2', 'type': 'dp', 'symbol': 'NSE:TCS-EQ'},
{'ltp': '4170.3', 'vol_traded_today': '2857851', 'last_traded_time': '1734687088', 'exch_feed_time': '1734687088', 'bid_size': '238', 'ask_size': '12', 'bid_price': '4168.7', 'ask_price': '4170.25', 'last_traded_qty': '2', 'tot_buy_qty': '89665', 'tot_sell_qty': '178175', 'avg_trade_price': '4270.56', 'low_price': '4160.8', 'high_price': '4382.0', 'lower_ckt': '0', 'upper_ckt': '0', 'open_price': '4329.0', 'prev_close_price': '4271.9', 'type': 'sf', 'symbol': 'NSE:TCS-EQ', 'ch': '-101.6', 'chp': '-2.3783'},
{'bid_price1': '4169.95', 'bid_price2': '4168.7', 'bid_price3': '4168.5', 'bid_price4': '4168.45', 'bid_price5': '4168.4', 'ask_price1': '4170.2', 'ask_price2': '4170.3', 'ask_price3': '4170.35', 'ask_price4': '4170.45', 'ask_price5': '4170.5', 'bid_size1': '46', 'bid_size2': '251', 'bid_size3': '210', 'bid_size4': '367', 'bid_size5': '959', 'ask_size1': '1', 'ask_size2': '5', 'ask_size3': '64', 'ask_size4': '63', 'ask_size5': '2', 'bid_order1': '2', 'bid_order2': '4', 'bid_order3': '1', 'bid_order4': '4', 'bid_order5': '4', 'ask_order1': '1', 'ask_order2': '3', 'ask_order3': '1', 'ask_order4': '2', 'ask_order5': '2', 'type': 'dp', 'symbol': 'NSE:TCS-EQ'},
{'ltp': '4170.2', 'vol_traded_today': '2857851', 'last_traded_time': '1734687088', 'exch_feed_time': '1734687089', 'bid_size': '46', 'ask_size': '1', 'bid_price': '4169.95', 'ask_price': '4170.2', 'last_traded_qty': '49', 'tot_buy_qty': '89761', 'tot_sell_qty': '175489', 'avg_trade_price': '4270.56', 'low_price': '4160.8', 'high_price': '4382.0', 'lower_ckt': '0', 'upper_ckt': '0', 'open_price': '4329.0', 'prev_close_price': '4271.9', 'type': 'sf', 'symbol': 'NSE:TCS-EQ', 'ch': '-101.7', 'chp': '-2.3807'},
]

In [9]:
for x in hello:
    r.xadd("NSE:TCS",x)

In [12]:
r.xread({"NSE:TCS":0})

[[b'NSE:TCS',
  [(b'1734768460305-0',
    {b'ltp': b'4181.5',
     b'vol_traded_today': b'2776701',
     b'last_traded_time': b'1734686963',
     b'exch_feed_time': b'1734686964',
     b'bid_size': b'22',
     b'ask_size': b'233',
     b'bid_price': b'4181.15',
     b'ask_price': b'4181.5',
     b'last_traded_qty': b'1',
     b'tot_buy_qty': b'92749',
     b'tot_sell_qty': b'186371',
     b'avg_trade_price': b'4273.32',
     b'low_price': b'4160.8',
     b'high_price': b'4382.0',
     b'lower_ckt': b'0',
     b'upper_ckt': b'0',
     b'open_price': b'4329.0',
     b'prev_close_price': b'4271.9',
     b'type': b'sf',
     b'symbol': b'NSE:TCS-EQ',
     b'ch': b'-90.4',
     b'chp': b'-2.1162'}),
   (b'1734768460306-0',
    {b'ltp': b'4181.5',
     b'vol_traded_today': b'2776795',
     b'last_traded_time': b'1734686964',
     b'exch_feed_time': b'1734686964',
     b'bid_size': b'4',
     b'ask_size': b'139',
     b'bid_price': b'4180.95',
     b'ask_price': b'4181.4',
     b'last_traded_